In [1]:
import torch.nn as nn
import torch

class SelfAttention_V2(nn.Module):

    def __init__(self, d_in, d_out, qkv_bias = False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out,bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out,bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out,bias=qkv_bias)
    

    def forward(self, x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1] ** 0.5 , dim = -1
        )

        context_vector = attn_weights @ values
        return context_vector

In [3]:
import torch

inputs = torch.tensor(
    [[0.43, 0.15, 0.89],  # your      (x^1)
     [0.55, 0.87, 0.66],  # journey   (x^2)
     [0.57, 0.85, 0.64],  # starts    (x^3)
     [0.22, 0.58, 0.34],  # with      (x^4)
     [0.77, 0.25, 0.10],  # one       (x^5)
     [0.05, 0.81, 0.55]]  # step      (x^6)
 )

In [4]:
d_in = inputs.shape[1]
d_out = 2

In [5]:
torch.manual_seed(789)
sa_v2 = SelfAttention_V2(d_in, d_out)
queries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs)

attn_scores = queries @ keys.T
attn_weights = torch.softmax(attn_scores / keys.shape[-1] ** 0.5, dim = 1)
print(attn_weights)

tensor([[0.1921, 0.1646, 0.1652, 0.1553, 0.1721, 0.1507],
        [0.2041, 0.1659, 0.1662, 0.1500, 0.1664, 0.1474],
        [0.2035, 0.1659, 0.1662, 0.1503, 0.1664, 0.1477],
        [0.1871, 0.1666, 0.1668, 0.1573, 0.1661, 0.1561],
        [0.1830, 0.1668, 0.1669, 0.1591, 0.1658, 0.1583],
        [0.1937, 0.1663, 0.1666, 0.1544, 0.1665, 0.1525]],
       grad_fn=<SoftmaxBackward0>)


In [15]:
context_length = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length,context_length))
mask_simple

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])

In [7]:
masked_atten_weight = attn_weights * mask_simple
masked_atten_weight

tensor([[0.1921, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2041, 0.1659, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2035, 0.1659, 0.1662, 0.0000, 0.0000, 0.0000],
        [0.1871, 0.1666, 0.1668, 0.1573, 0.0000, 0.0000],
        [0.1830, 0.1668, 0.1669, 0.1591, 0.1658, 0.0000],
        [0.1937, 0.1663, 0.1666, 0.1544, 0.1665, 0.1525]],
       grad_fn=<MulBackward0>)

In [10]:
row_sums = masked_atten_weight.sum(dim = 1 , keepdim=True)
masked_simple_norm = masked_atten_weight / row_sums
print(masked_simple_norm)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2760, 0.2459, 0.2461, 0.2321, 0.0000, 0.0000],
        [0.2175, 0.1982, 0.1983, 0.1890, 0.1970, 0.0000],
        [0.1937, 0.1663, 0.1666, 0.1544, 0.1665, 0.1525]],
       grad_fn=<DivBackward0>)


In [11]:
torch.manual_seed(789)
sa_v2 = SelfAttention_V2(d_in, d_out)
queries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs)

attn_scores = queries @ keys.T

In [ ]:
cmasked


tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2661, 0.1026, 0.1040, 0.0209,   -inf,   -inf],
        [0.2183, 0.0874, 0.0882, 0.0198, 0.0786,   -inf],
        [0.3436, 0.1284, 0.1304, 0.0235, 0.1298, 0.0061]],
       grad_fn=<MaskedFillBackward0>)

In [16]:
attn_weights = torch.softmax(masked / keys.shape[-1]**0.5 , dim = 1)
attn_weights

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2760, 0.2459, 0.2461, 0.2321, 0.0000, 0.0000],
        [0.2175, 0.1982, 0.1983, 0.1890, 0.1970, 0.0000],
        [0.1937, 0.1663, 0.1666, 0.1544, 0.1665, 0.1525]],
       grad_fn=<SoftmaxBackward0>)

In [17]:
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5)
example = torch.ones(6,6)
print(dropout(example))

tensor([[2., 2., 0., 2., 2., 0.],
        [0., 0., 0., 2., 0., 2.],
        [2., 2., 2., 2., 0., 2.],
        [0., 2., 2., 0., 0., 2.],
        [0., 2., 0., 2., 0., 2.],
        [0., 2., 2., 2., 2., 0.]])


In [18]:
print(dropout(attn_weights))

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.6194, 0.6206, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.4922, 0.4641, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.3940, 0.0000],
        [0.3873, 0.3326, 0.0000, 0.3089, 0.3330, 0.3051]],
       grad_fn=<MulBackward0>)


<div class="alert alert-block alert-info">
Implementing a COMPACT CAUSAL SELF ATTENTION CLASS
</div>

In [24]:
import torch.nn as nn
import torch

class CasualAttentionBlock(nn.Module):

    def __init__(self, d_in, d_out,context_length,dropout, qkv_bias = False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out,bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out,bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out,bias=qkv_bias)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer('mask', torch.triu(torch.ones(context_length,context_length),diagonal=1))
    


    def forward(self, x):
        b, num_tokens , d_in = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1,2)
        attn_scores.masked_fill(
            self.mask.bool()[:num_tokens,:num_tokens],
            -torch.inf
        )
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1] ** 0.5 , dim = -1
        )

        context_vector = attn_weights @ values
        return context_vector

In [25]:
batch = torch.stack((inputs,inputs), dim = 0)

In [26]:
torch.manual_seed(123)
context_length = batch.shape[1]


ca = CasualAttentionBlock(d_in , d_out, context_length, dropout=0.2)

context_vector = ca(batch)
context_vector



tensor([[[-0.5346, -0.1052],
         [-0.5332, -0.1081],
         [-0.5332, -0.1081],
         [-0.5307, -0.1077],
         [-0.5319, -0.1067],
         [-0.5308, -0.1082]],

        [[-0.5346, -0.1052],
         [-0.5332, -0.1081],
         [-0.5332, -0.1081],
         [-0.5307, -0.1077],
         [-0.5319, -0.1067],
         [-0.5308, -0.1082]]], grad_fn=<UnsafeViewBackward0>)